![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FDev&file=tensorboard_in_notebooks+-+vetex-ai-tensorboard.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Dev/tensorboard_in_notebooks%20-%20vetex-ai-tensorboard.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FDev%2Ftensorboard_in_notebooks%2520-%2520vetex-ai-tensorboard.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Dev/tensorboard_in_notebooks%20-%20vetex-ai-tensorboard.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Dev/tensorboard_in_notebooks%20-%20vetex-ai-tensorboard.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Using TensorBoard in Notebooks

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/tensorboard_in_notebooks.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorboard/blob/master/docs/tensorboard_in_notebooks.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/tensorboard/docs/tensorboard_in_notebooks.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

TensorBoard can be used directly within notebook experiences such as [Colab](https://colab.research.google.com/) and [Jupyter](https://jupyter.org/). This can be helpful for sharing results, integrating TensorBoard into existing workflows, and using TensorBoard without installing anything locally.

## Setup

Start by installing TF 2.0 and loading the TensorBoard notebook extension:

**For Jupyter users:** If you’ve installed Jupyter and TensorBoard into
the same virtualenv, then you should be good to go. If you’re using a
more complicated setup, like a global Jupyter installation and kernels
for different Conda/virtualenv environments, then you must ensure that
the `tensorboard` binary is on your `PATH` inside the Jupyter notebook
context. One way to do this is to modify the `kernel_spec` to prepend
the environment’s `bin` directory to `PATH`, [as described here][1].

[1]: https://github.com/ipython/ipykernel/issues/395#issuecomment-479787997


**For Docker users:** In case you are running a [Docker](https://docs.docker.com/install/) image of [Jupyter Notebook server using TensorFlow's nightly](https://www.tensorflow.org/install/docker#examples_using_cpu-only_images), it is necessary to expose not only the notebook's port, but the TensorBoard's port. Thus, run the container with the following command:

```
docker run -it -p 8888:8888 -p 6006:6006 \
tensorflow/tensorflow:nightly-py3-jupyter
```

where the `-p 6006` is the default port of TensorBoard. This will allocate a port for you to run one TensorBoard instance. To have concurrent instances, it is necessary to allocate more ports. Also, pass `--bind_all` to `%tensorboard` to expose the port outside the container.

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

Import TensorFlow, datetime, and os:

In [ ]:
import tensorflow as tf
import datetime, os

## TensorBoard in notebooks

Download the [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist) dataset and scale it:

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

4422102/4422102 [==============================] - 0s 0us/step


Create a very simple model:

In [ ]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28), name='layers_flatten'),
    tf.keras.layers.Dense(512, activation='relu', name='layers_dense'),
    tf.keras.layers.Dropout(0.2, name='layers_dropout'),
    tf.keras.layers.Dense(10, activation='softmax', name='layers_dense_2')
  ])

Train the model using Keras and the TensorBoard callback:

In [ ]:
def train_model():

  model = create_model()
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(x=x_train,
            y=y_train,
            epochs=5,
            validation_data=(x_test, y_test),
            callbacks=[tensorboard_callback])

train_model()

Epoch 1/5
1875/1875 [==============================] - 14s 6ms/step - loss: 0.4965 - accuracy: 0.8224 - val_loss: 0.4227 - val_accuracy: 0.8465
Epoch 2/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.3800 - accuracy: 0.8608 - val_loss: 0.3993 - val_accuracy: 0.8555
Epoch 3/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.3512 - accuracy: 0.8706 - val_loss: 0.3581 - val_accuracy: 0.8699
Epoch 4/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.3300 - accuracy: 0.8785 - val_loss: 0.3384 - val_accuracy: 0.8771
Epoch 5/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.3122 - accuracy: 0.8845 - val_loss: 0.3542 - val_accuracy: 0.8740


---

## Try Vertex AI TensorBoard

https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview#continuous-monitoring

`<START TRY VERTEX AI TENSORBOARD>`

In [ ]:
if "google.colab" in str(get_ipython()):
    from google.colab import auth as google_auth
    google_auth.authenticate_user()

In [ ]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [ ]:
REGION = 'us-central1'
TB_NAME = 'COLAB_TEST'
EXPERIMENT = 'colab-enterprise'
EXPERIMENT_RUN = 'run-'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [ ]:
from google.cloud import aiplatform
aiplatform.init(project = PROJECT_ID, location = REGION)

In [ ]:
# create or recall TB Instance:
tb = aiplatform.Tensorboard.list(filter=f"display_name={TB_NAME}")
if tb:
    tb = tb[0]
else:
    tb = aiplatform.Tensorboard.create(display_name = TB_NAME)

INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:Creating Tensorboard
INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:Create Tensorboard backing LRO: projects/1026793852137/locations/us-central1/tensorboards/297510254290141184/operations/3308249824473645056
INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:Tensorboard created. Resource name: projects/1026793852137/locations/us-central1/tensorboards/297510254290141184
INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:To use this Tensorboard in another session:
INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:tb = aiplatform.Tensorboard('projects/1026793852137/locations/us-central1/tensorboards/297510254290141184')


In [ ]:
# one-time logging
aiplatform.upload_tb_log(
    tensorboard_id = tb.name,
    tensorboard_experiment_name = EXPERIMENT,
    experiment_display_name = EXPERIMENT,
    description = EXPERIMENT,
    run_name_prefix = EXPERIMENT_RUN,
    logdir = 'logs'
)

View your Tensorboard at https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+297510254290141184+experiments+colab-enterprise
[2023-09-29T11:17:44] Started scanning logdir.
[2023-09-29T11:17:45] Total uploaded: 30 scalars, 40 tensors (14.4 kB), 1 binary objects (42.1 kB)


INFO:google.cloud.aiplatform.tensorboard.uploader_tracker:One time TensorBoard log upload completed.


In [ ]:
# continous logging: start and stop around the training routine
aiplatform.start_upload_tb_log(
    tensorboard_id = tb.name,
    tensorboard_experiment_name = EXPERIMENT,
    experiment_display_name = EXPERIMENT,
    description = EXPERIMENT,
    run_name_prefix = EXPERIMENT_RUN,
    logdir = 'logs'
)

View your Tensorboard at https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+297510254290141184+experiments+colab-enterprise


In [ ]:
train_model()

Epoch 1/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.4955 - accuracy: 0.8225 - val_loss: 0.4011 - val_accuracy: 0.8530
Epoch 2/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.3822 - accuracy: 0.8600 - val_loss: 0.3896 - val_accuracy: 0.8550
Epoch 3/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.3478 - accuracy: 0.8708 - val_loss: 0.3905 - val_accuracy: 0.8549
Epoch 4/5
1875/1875 [==============================] - 10s 6ms/step - loss: 0.3313 - accuracy: 0.8795 - val_loss: 0.3476 - val_accuracy: 0.8737
Epoch 5/5
1875/1875 [==============================] - 10s 6ms/step - loss: 0.3100 - accuracy: 0.8855 - val_loss: 0.3420 - val_accuracy: 0.8789


In [ ]:
aiplatform.end_upload_tb_log()

In [15]:
# delete experiment from tensorboard instance:
for e in aiplatform.TensorboardExperiment.list(tensorboard_name = tb.resource_name):
  print(e.name)
  e.delete()

INFO:google.cloud.aiplatform.base:Deleting TensorboardExperiment : projects/1026793852137/locations/us-central1/tensorboards/297510254290141184/experiments/colab-enterprise
INFO:google.cloud.aiplatform.base:Delete TensorboardExperiment  backing LRO: projects/1026793852137/locations/us-central1/tensorboards/297510254290141184/operations/3374677918977359872
INFO:google.cloud.aiplatform.base:TensorboardExperiment deleted. . Resource name: projects/1026793852137/locations/us-central1/tensorboards/297510254290141184/experiments/colab-enterprise


colab-enterprise


In [16]:
# delete tensorboard instance:
for b in aiplatform.Tensorboard.list(filter = f"display_name={TB_NAME}"):
  print(b.display_name)
  b.delete()

INFO:google.cloud.aiplatform.base:Deleting Tensorboard : projects/1026793852137/locations/us-central1/tensorboards/297510254290141184
INFO:google.cloud.aiplatform.base:Delete Tensorboard  backing LRO: projects/1026793852137/locations/us-central1/operations/5135585373279223808
INFO:google.cloud.aiplatform.base:Tensorboard deleted. . Resource name: projects/1026793852137/locations/us-central1/tensorboards/297510254290141184


COLAB_TEST



`<END TRY VERTEX AI TENSORBOARD>`

---

Start TensorBoard within the notebook using [magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html):

In [ ]:
%tensorboard --logdir logs

<IPython.core.display.Javascript object>

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/notebook_tensorboard.png?raw=1"/> -->

You can now view dashboards such as **Time Series**, **Graphs**, **Distributions**, and others. Some dashboards are not available yet in Colab (such as the profile plugin).

The `%tensorboard` magic has exactly the same format as the TensorBoard command line invocation, but with a `%`-sign in front of it.

You can also start TensorBoard before training to monitor it in progress:

In [ ]:
%tensorboard --logdir logs

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/notebook_tensorboard_two_runs.png?raw=1"/> -->

The same TensorBoard backend is reused by issuing the same command. If a different logs directory was chosen, a new instance of TensorBoard would be opened. Ports are managed automatically.

Start training a new model and watch TensorBoard update automatically every 30 seconds or refresh it with the button on the top right:

In [ ]:
train_model()

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 11s 184us/sample - loss: 0.4968 - accuracy: 0.8223 - val_loss: 0.4216 - val_accuracy: 0.8481
Epoch 2/5
60000/60000 [==============================] - 11s 176us/sample - loss: 0.3847 - accuracy: 0.8587 - val_loss: 0.4056 - val_accuracy: 0.8545
Epoch 3/5
60000/60000 [==============================] - 11s 176us/sample - loss: 0.3495 - accuracy: 0.8727 - val_loss: 0.3600 - val_accuracy: 0.8700
Epoch 4/5
60000/60000 [==============================] - 11s 179us/sample - loss: 0.3282 - accuracy: 0.8795 - val_loss: 0.3636 - val_accuracy: 0.8694
Epoch 5/5
60000/60000 [==============================] - 11s 176us/sample - loss: 0.3115 - accuracy: 0.8839 - val_loss: 0.3438 - val_accuracy: 0.8764


You can use the `tensorboard.notebook` APIs for a bit more control:

In [ ]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

Known TensorBoard instances:
  - port 6006: logdir logs (started 0:00:54 ago; pid 265)


In [ ]:
# Control TensorBoard display. If no port is provided,
# the most recently launched TensorBoard is used
notebook.display(port=6006, height=1000)

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/notebook_tensorboard_tall.png?raw=1"/> -->